In [1]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [2]:
train = pd.read_csv('train.csv')

test = pd.read_csv('test.csv')

In [3]:
train['Type'] = 'Train' 
test['Type'] = 'Test'
fullData = pd.concat([train,test],axis=0)

/home/paologio/anaconda3/envs/venv/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


In [4]:
#Identifying ID, Categorical
ID_col = ['User_ID','Product_ID']
flag_col= ['Type']
target_col = ["Purchase"]
cat_cols= ['Gender','Age','City_Category','Stay_In_Current_City_Years']
num_cols= list(set(list(fullData.columns))-set(cat_cols)-set(ID_col)-set(target_col)-set(flag_col))

# Combined numerical and Categorical variables
num_cat_cols = num_cols+cat_cols

#Create a new variable for each variable having missing value with VariableName_NA 
# and flag missing value with 1 and other with 0

for var in num_cat_cols:
    if fullData[var].isnull().any()==True:
        fullData[var+'_NA']=fullData[var].isnull()*1

#Impute numerical missing values with mean
fullData[num_cols] = fullData[num_cols].fillna(fullData[num_cols].mean())

#Impute categorical missing values with -9999
fullData[cat_cols] = fullData[cat_cols].fillna(value = -9999)

In [5]:
for var in cat_cols:
    number = LabelEncoder()
    fullData[var] = number.fit_transform(fullData[var].astype('str'))

In [6]:
features = ['Gender',
 'Stay_In_Current_City_Years',
 'Product_Category_2_NA',
 'Product_Category_2',
 'Product_Category_3_NA',
 'Product_Category_3',
 'City_Category',
 'Age',
 'Occupation',
 'Marital_Status',
 'Product_Category_1']
fullData[features] = fullData[features]/fullData[features].max()

In [17]:
train=fullData[fullData['Type']=='Train']
test=fullData[fullData['Type']=='Test']
features=list(set(list(fullData.columns))-set(ID_col)-set(target_col)-set(flag_col))

X = train[features].values
y = train[target_col].values

print(train['Purchase'].values)

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.30, random_state=42)

[ 8370. 15200.  1422. ...   137.   365.   490.]


In [18]:
y_train

array([[ 3886.],
       [12148.],
       [ 8054.],
       ...,
       [ 6954.],
       [ 6151.],
       [ 7146.]])

In [8]:
model = Sequential()
model.add(Dense(100, input_dim=11, activation= "relu"))
model.add(Dense(50, activation= "relu"))
model.add(Dense(1))
model.summary() #Print model Summary

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 100)               1200      
_________________________________________________________________
dense_2 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 51        
Total params: 6,301
Trainable params: 6,301
Non-trainable params: 0
_________________________________________________________________


In [9]:
model.compile(loss= "mean_squared_error" , optimizer="adam", metrics=["mean_squared_error"])

In [10]:
model.fit(X_train, y_train, epochs=10)

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
385047/385047 [==============================] - 11s 28us/step - loss: 25016308.3974 - mean_squared_error: 25016308.3974
Epoch 2/10
385047/385047 [==============================] - 10s 27us/step - loss: 19613706.3479 - mean_squared_error: 19613706.3479
Epoch 3/10
385047/385047 [==============================] - 11s 28us/step - loss: 17912092.1678 - mean_squared_error: 17912092.1678
Epoch 4/10
385047/385047 [==============================] - 10s 27us/step - loss: 17708141.1281 - mean_squared_error: 17708141.1281
Epoch 5/10
385047/385047 [==============================] - 11s 29us/step - loss: 17694560.4272 - mean_squared_error: 17694560.4272
Epoch 6/10
385047/385047 [==============================] - 13s 33us/step - loss: 17686294.6435 - mean_squared_error: 17686294.6435
Epoch 7/10
385047/385047 [==============================] - 12s 32us/step - loss: 17683936.0766 - mean_squared_error: 17683936.0766
Epoch 8/10
385047/385047 [==

In [11]:
pred= model.predict(X_valid)
score = np.sqrt(mean_squared_error(y_valid,pred))

score

4208.540211639948

In [13]:
model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))

Train on 385047 samples, validate on 165021 samples
Epoch 1/10
385047/385047 [==============================] - 12s 32us/step - loss: 17678627.8286 - mean_squared_error: 17678627.8286 - val_loss: 17699393.8302 - val_mean_squared_error: 17699393.8302
Epoch 2/10
385047/385047 [==============================] - 12s 32us/step - loss: 17678606.5361 - mean_squared_error: 17678606.5361 - val_loss: 17770353.2717 - val_mean_squared_error: 17770353.2717
Epoch 3/10
385047/385047 [==============================] - 12s 31us/step - loss: 17678685.6964 - mean_squared_error: 17678685.6964 - val_loss: 17735804.8381 - val_mean_squared_error: 17735804.8381
Epoch 4/10
191616/385047 [=============>................] - ETA: 5s - loss: 17573355.9173 - mean_squared_error: 17573355.9173

KeyboardInterrupt: 

In [12]:
X_test = test[features].values

y_test = model.predict(X_test)

y_test

array([[13765.715],
       [ 9354.206],
       [ 7190.423],
       ...,
       [13859.009],
       [ 8085.722],
       [ 7162.247]], dtype=float32)